In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [21]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool, FunctionTool,google_search
from google.genai import types

In [22]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [23]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [24]:
# 1. Input Collector
input_collector = Agent(
        name="input_collector",
        model=Gemini(model_name="gemini-1.5-flash"),
        description="Collects user preferences for mess meal planning.",
        instruction="""
        Ask the user step-by-step:
        - How many people? (default 4)
        - Weekly budget per person? (default ₹400)
        - Max non-veg times a week? (0–14)
        - Preferred non-veg: chicken, egg, fish, mutton
        - Favourite veg dishes
        - Any dish you HATE?
        - Jain / allergies?
        - Breakfast from mess fund?
        Also ask:
        - Goal
        - Health conditions
        - Activity level
        Return clean JSON.
        """
)     

# 2. Constraint Validator
validator = Agent(
        name="constraint_validator",
        model=Gemini(model_name="gemini-1.5-flash"),
        description="Applies hard mess rules + user rules.",
        instruction="""
        Hard rules (₹400/person, 4 people):
        - Adjust non-veg if over budget
        - Only 1 non-veg meal/day
        - No paneer if budget ≤ ₹400
        - Weekdays cooking ≤60 min
        - Sunday = special
        - Breakfast 4–5 days allowed
        Validate and return JSON.
        """
)
# 3. Meal Planner
planner = Agent(
        name="meal_planner",
        model=Gemini(model_name="gemini-1.5-flash"),
        description="Generates full 7-day mess menu.",
        instruction="""
        Create a 7-day 2-course meal plan.
        - Plan 14 meals, not daywise
        - Sunday special
        - Lunch must include rice
        - Chicken/fish at least once/week
        - No consecutive identical meals
        - Stay in budget, use leftovers
        Return markdown table + cost.
        """
)

# Feedback Agent
feedback_agent = Agent(
        name="feedback",
        model=Gemini(model_name="gemini-1.5-flash"),
        instruction="Ask for rating (1-5) and what changes are needed. Return JSON."
)


budget_agent = Agent(
        name="budget_estimator",
        model=Gemini(model_name="gemini-1.5-flash"),
        description="Real-time grocery prices using Google search",
        instruction="""
        Use Google Search to find ingredient prices.
        Compute exact weekly cost.
        Suggest cheaper alternatives.
        Return final menu + cost.
        """,
        tools=[google_search]
)
    
# Root agent
root_agent = Agent(
        name="MessMealPlanner",
        model=Gemini(model_name="gemini-1.5-flash"),
        description="Full Indian mess meal planner",
        instruction="Run in order: input_collector → constraint_validator → meal_planner.",
        sub_agents=[input_collector, validator, planner, feedback_agent]
)


In [25]:
runner = InMemoryRunner(
    agent=root_agent,
    app_name="mess_planner"
)

print("✅ Runner created.")

✅ Runner created.


In [26]:
response = await runner.run_debug("""
generate a weekly mess food routine for our mess.

Here are my preferences:
- People: 4
- Weekly budget: 400 per person
- Max non-veg times:5 
- Preferred non-veg: egg, chicken, fish 
- Favourite veg:  aloo,dal,paneer
- Hate: karela
- egg allergies
- Breakfast included: no
"""
)


 ### Created new session: debug_session_id

User > 
generate a weekly mess food routine for our mess.

Here are my preferences:
- People: 4
- Weekly budget: 400 per person
- Max non-veg times:5 
- Preferred non-veg: egg, chicken, fish 
- Favourite veg:  aloo,dal,paneer
- Hate: karela
- egg allergies
- Breakfast included: no



meal_planner > Here is a 7-day, 2-course mess meal plan, designed considering your preferences and constraints. Please note that the weekly budget of ₹400 per person (approximately ₹28.57 per meal per person for 14 meals) is extremely challenging for a 2-course meal, especially when including non-vegetarian options. The plan below is highly economical and focuses on staple ingredients and simple preparations. Even with these considerations, the estimated cost slightly exceeds your stated budget.

**Meal Plan for 4 People (7 Days, 2 Courses/Meal)**

| Day       | Meal     | Course 1 (Main Dish)    | Course 2 (Side/Accompaniment, Rice for Lunch) | Estimated Cost Per Person (₹) |
| :-------- | :------- | :---------------------- | :------------------------------------------- | :---------------------------- |
| **Monday**| Lunch    | Dal Tadka               | Rice & Pickle                                | 25                            |
|           | Dinner   | Aloo Bhujia             | Rot